# What Can You Expect In this Notebook?

## Goals
1. Create a collaborative filtering feature
1. Make book clustering

In [1]:
# import libraries (you may add additional imports but you may not have to)
import logging
logging.captureWarnings(True)

import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)
from scipy.sparse import coo_matrix
from scipy.sparse import csr_matrix

#viz lib
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import seaborn as sns

#
from sklearn.metrics.pairwise import cosine_similarity

# #to scale the data using z-score
# from sklearn.preprocessing import StandardScaler

# #Importing PCA
# from sklearn.decomposition import PCA

#
# from surprise import Reader

#
import dill as pickle

In [2]:
df = pd.read_csv("data/clean_data.csv")

In [3]:
df.head()

,isbn,book_title,book_author,year_of_publication,publisher,mod_title,isbn_index,user_id,book_rating,location,age
0,0440234743,The Testament,John Grisham,1999,Dell,the testament,87548,277478.0,0.0,"schiedam, zuid-holland, netherlands",31.0
1,0440234743,The Testament,John Grisham,1999,Dell,the testament,87548,2977.0,0.0,"richland, washington, usa",25.0
2,0440234743,The Testament,John Grisham,1999,Dell,the testament,87548,3363.0,0.0,"knoxville, tennessee, usa",29.0
3,0440234743,The Testament,John Grisham,1999,Dell,the testament,87548,7346.0,9.0,"sunnyvale, california, usa",49.0
4,0440234743,The Testament,John Grisham,1999,Dell,the testament,87548,9747.0,0.0,"o`fallon, missouri, usa",24.0


In [4]:
#User-Book matrix
user_book_df = pd.pivot_table(
    data = df,
    index="isbn_index", 
    columns="user_id", 
    values="book_rating"
    ).sort_index()

# Some of the books have 0 rating, indicating that the books are liked by certain users but never rated. 
# We need to differentiate liked books and rated books.
# We add 1 to the rating, and fill null value with 0 (liked but unrated)
user_book_df += 1

user_book_df.fillna(0, inplace = True)

user_book_df.head()

user_id,254.0,507.0,882.0,1424.0,1435.0,1733.0,1903.0,2033.0,2110.0,2276.0,...,275020.0,275970.0,276463.0,276680.0,277427.0,277478.0,277639.0,278137.0,278188.0,278418.0
isbn_index,,,,,,,,,,,,,,,,,,,,,
806,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1336,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1472,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1769,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# convert the dataframe to sparse matrix
user_book_sparse = csr_matrix(user_book_df.values)

In [6]:
# calculate the cosine similarity between books according to ratings from users
similarity_scores = cosine_similarity(user_book_sparse)
similarity_scores.shape

(2161, 2161)

In [7]:
# create a mapping dictionary
sparse_to_df_map = {j:i for i,j in zip(range(user_book_sparse.shape[0]),user_book_df.index.unique())}

In [8]:
# make a function to recommend the books
def recommend(isbn_index):
    # index fetch
    index = np.where(user_book_df.index==isbn_index)[0][0]
    similar_items = sorted(list(enumerate(similarity_scores[index])),key=lambda x:x[1],reverse=True)[0:6]
    
    data = [user_book_df.index[i[0]] for i in similar_items]
    
    return data

In [9]:
df[df['isbn_index'].isin(recommend(806))][["isbn","book_title","book_author","year_of_publication"]].drop_duplicates()

,isbn,book_title,book_author,year_of_publication
22196,000649840X,Angelas Ashes,Frank Mccourt,0
43809,1844262553,Free,Paul Vincent,2003
70750,0330267388,"Life,the Universe and Everything",Douglas Adams,0
76110,0684859734,Grave Secrets (Temperance Brennan Novel (Hardcover)),Kathy Reichs,2002
81841,3423202327,MÃ?Â¶rder ohne Gesicht.,Henning Mankell,1999
86921,0375758232,Paris to the Moon,Adam Gopnik,2001


## PICKLE RIICKKKK!!!

In [10]:
# make a function to recommend the books
def new_recommend(isbn_index):
    index = sparse_to_df_map[isbn_index]
    similar_items = sorted(list(enumerate(similarity_scores[index])),key=lambda x:x[1],reverse=True)[0:6]
    
    data = [list(sparse_to_df_map.keys())[list(sparse_to_df_map.values()).index(i[0])] for i in similar_items]
    
    return data

In [11]:
pickle.dump(similarity_scores, open('pickles/similarity_scores.pkl','wb'))
pickle.dump(sparse_to_df_map, open('pickles/sparse_to_df_map.pkl','wb'))
pickle.dump(new_recommend, open('pickles/recommender.pkl','wb'))

In [12]:
sim_score = pickle.load(open('pickles/similarity_scores.pkl', 'rb'))
sparse_to_df_map = pickle.load(open('pickles/sparse_to_df_map.pkl', 'rb'))
recommender = pickle.load(open('pickles/recommender.pkl', 'rb'))

In [13]:
_ = df[df['isbn_index'].isin(new_recommend(806))][["isbn","isbn_index","book_title","book_author","year_of_publication"]].drop_duplicates()
_ = _[_['isbn_index']!=806]
_

,isbn,isbn_index,book_title,book_author,year_of_publication
43809,1844262553,237841,Free,Paul Vincent,2003
70750,0330267388,39553,"Life,the Universe and Everything",Douglas Adams,0
76110,0684859734,148708,Grave Secrets (Temperance Brennan Novel (Hardcover)),Kathy Reichs,2002
81841,3423202327,254702,MÃ?Â¶rder ohne Gesicht.,Henning Mankell,1999
86921,0375758232,62606,Paris to the Moon,Adam Gopnik,2001


In [14]:
import sys
sys.getsizeof(sparse_to_df_map)

73816